In [40]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_core.prompts import PromptTemplate
from IPython.display import display, Markdown

## Step 1a - Indexing (Document Ingestion)

In [3]:
video_id = "v4L80iWDwoI" # only the ID, not full URL (https://www.youtube.com/watch?v=v4L80iWDwoI)

try:
    # If you don’t care which language, this returns the “best” one
    transcript_list = YouTubeTranscriptApi().fetch(video_id, languages=['en'])
    
    # Flatten it to plain text
    transcript = " ".join(chunk.text for chunk in transcript_list)
    print(transcript[:50])
except TranscriptsDisabled:
    print("No captions available for this video.")

all praise and thanks belongs to Allah subhanahu w


## Step 1b - Indexing (Text Splitting)

In [11]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])
print(f"Number of chunks: {len(chunks)}")

Number of chunks: 34


In [15]:
chunks[0]

Document(metadata={}, page_content="all praise and thanks belongs to Allah subhanahu waala and May the peace and blessing of Allah be upon his servant and final messenger Muhammad s. wasam firstly I'm going to address you as brothers and sisters so I say my dear brothers and sisters in ISL and of course we also thank our D teacher F mallah we learned so much from him when we were in school and Subhan Allah he is still teaching at the school and this requires a lot of dedication a lot of patience a lot of perseverance and so we ask Allah to accept his efforts and of course the efforts of all your teachers and we ask him waala to accept from us all and to bless our deeds and to grant us the highest levels of the paradise say aan bye okay brothers and sisters I'm going to ask the brothers a question who is willing to volunteer to answer my question what's your name Aiden Aiden brother Aiden yes come here I'm going to ask you a question and I want an answer that is real and positive from y

## Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)

In [18]:
embeddings = HuggingFaceEmbeddings(
                model_name="BAAI/bge-small-en-v1.5",
                model_kwargs={"device": "cpu"},
                encode_kwargs={"normalize_embeddings": True}
            )
vector_store = FAISS.from_documents(chunks, embeddings)

In [21]:
print(f"Random embedding vector: {vector_store.index_to_docstore_id[0]}")

Random embedding vector: 06438360-a17c-4f9f-a5d3-36e78b71d615


In [20]:
print(f"Number of embeddings vectors: {len(vector_store.index_to_docstore_id)}")

Number of embeddings vectors: 34


In [24]:
print(f" Document: {vector_store.get_by_ids(['06438360-a17c-4f9f-a5d3-36e78b71d615'])}")

 Document: [Document(id='06438360-a17c-4f9f-a5d3-36e78b71d615', metadata={}, page_content="all praise and thanks belongs to Allah subhanahu waala and May the peace and blessing of Allah be upon his servant and final messenger Muhammad s. wasam firstly I'm going to address you as brothers and sisters so I say my dear brothers and sisters in ISL and of course we also thank our D teacher F mallah we learned so much from him when we were in school and Subhan Allah he is still teaching at the school and this requires a lot of dedication a lot of patience a lot of perseverance and so we ask Allah to accept his efforts and of course the efforts of all your teachers and we ask him waala to accept from us all and to bless our deeds and to grant us the highest levels of the paradise say aan bye okay brothers and sisters I'm going to ask the brothers a question who is willing to volunteer to answer my question what's your name Aiden Aiden brother Aiden yes come here I'm going to ask you a questio

## Step 2 - Retrieval

In [28]:
retreiver = vector_store.as_retriever(search_type="mmr", search_kwargs={"fetch_k": 5, "k": 2})
retreiver

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023DA068D2E0>, search_type='mmr', search_kwargs={'fetch_k': 5, 'k': 2})

In [29]:
retreiver.invoke("why hijab tells?")

[Document(id='361951f6-4274-404e-b06f-eeedb593ae5b', metadata={}, page_content="is saying no if you wear the hijab you will be recognized what does that mean it means you'll be recognized as a Muslim woman and if you're recognized as a Muslim woman that comes with guidelines meaning I don't entertain indecency I don't entertain Al what I'm wearing sends a very powerful and clear message to whoever is creeping to me intending Haram this clothing of mine helps me repel these forces of evil that are attacking me keeps them away from me that's what Al does that's why Allah said and once you're recognized as a Muslim woman they will not be harmed you see that that's why my sister you need to ad to all your life and you need to see it as an act of kindness and mercy from Allah upon you it is not restricting your freedom it is not holding you back from anything rather it is giving you every means of success in this life and in the afterlife despite its difficulty immense reward is given on th

## Step 3 - Augmentation

In [30]:
llm = ChatGroq(model="openai/gpt-oss-20b", temperature=0.2)

In [32]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [34]:
query = "what is hijab? what it shows?"
retreived_docs = retreiver.invoke(query)
retreived_docs[0]

Document(id='361951f6-4274-404e-b06f-eeedb593ae5b', metadata={}, page_content="is saying no if you wear the hijab you will be recognized what does that mean it means you'll be recognized as a Muslim woman and if you're recognized as a Muslim woman that comes with guidelines meaning I don't entertain indecency I don't entertain Al what I'm wearing sends a very powerful and clear message to whoever is creeping to me intending Haram this clothing of mine helps me repel these forces of evil that are attacking me keeps them away from me that's what Al does that's why Allah said and once you're recognized as a Muslim woman they will not be harmed you see that that's why my sister you need to ad to all your life and you need to see it as an act of kindness and mercy from Allah upon you it is not restricting your freedom it is not holding you back from anything rather it is giving you every means of success in this life and in the afterlife despite its difficulty immense reward is given on the

In [36]:
context_text = "\n\n".join(doc.page_content for doc in retreived_docs)
context_text

"is saying no if you wear the hijab you will be recognized what does that mean it means you'll be recognized as a Muslim woman and if you're recognized as a Muslim woman that comes with guidelines meaning I don't entertain indecency I don't entertain Al what I'm wearing sends a very powerful and clear message to whoever is creeping to me intending Haram this clothing of mine helps me repel these forces of evil that are attacking me keeps them away from me that's what Al does that's why Allah said and once you're recognized as a Muslim woman they will not be harmed you see that that's why my sister you need to ad to all your life and you need to see it as an act of kindness and mercy from Allah upon you it is not restricting your freedom it is not holding you back from anything rather it is giving you every means of success in this life and in the afterlife despite its difficulty immense reward is given on the day of judgment and you have so many wom ahead of you and behind you in\n\nth

In [37]:
final_prompt = prompt.format(context=context_text, question=query)
print(final_prompt)


      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      is saying no if you wear the hijab you will be recognized what does that mean it means you'll be recognized as a Muslim woman and if you're recognized as a Muslim woman that comes with guidelines meaning I don't entertain indecency I don't entertain Al what I'm wearing sends a very powerful and clear message to whoever is creeping to me intending Haram this clothing of mine helps me repel these forces of evil that are attacking me keeps them away from me that's what Al does that's why Allah said and once you're recognized as a Muslim woman they will not be harmed you see that that's why my sister you need to ad to all your life and you need to see it as an act of kindness and mercy from Allah upon you it is not restricting your freedom it is not holding you back from anything rather it is giving you every means of success in t

## Step 4 - Generation

In [44]:
answer = llm.invoke(final_prompt)
Markdown(answer.content)

**Hijab** is the modest clothing that a Muslim woman wears.  
According to the transcript, it:

* **Identifies her as a Muslim woman** – the garment signals to others that she follows Islamic modesty guidelines.  
* **Repels unwanted attention** – by being recognized as a Muslim, it discourages “creeping” or harassing behavior.  
* **Acts as a protective shield** – the hijab is described as a means that keeps “forces of evil” away and protects her from harm.  
* **Shows obedience to Allah’s command** – it is a sign of following the divine instruction to dress modestly, which brings spiritual reward and safety.

## Building a Chain

In [45]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [46]:
def format_docs(retreived_docs):
    context_docs = "\n\n".join(doc.page_content for doc in retreived_docs)
    return context_docs

In [47]:
parallel_chain = RunnableParallel({
    "context": retreiver | RunnableLambda(format_docs),
    "question": RunnablePassthrough()
})

In [48]:
parallel_chain.invoke(query)

{'context': "is saying no if you wear the hijab you will be recognized what does that mean it means you'll be recognized as a Muslim woman and if you're recognized as a Muslim woman that comes with guidelines meaning I don't entertain indecency I don't entertain Al what I'm wearing sends a very powerful and clear message to whoever is creeping to me intending Haram this clothing of mine helps me repel these forces of evil that are attacking me keeps them away from me that's what Al does that's why Allah said and once you're recognized as a Muslim woman they will not be harmed you see that that's why my sister you need to ad to all your life and you need to see it as an act of kindness and mercy from Allah upon you it is not restricting your freedom it is not holding you back from anything rather it is giving you every means of success in this life and in the afterlife despite its difficulty immense reward is given on the day of judgment and you have so many wom ahead of you and behind 

In [50]:
parser = StrOutputParser()
main_chain = parallel_chain | prompt | llm | parser
output = main_chain.invoke(query)
Markdown(output)

**Hijab** is the modest covering that a Muslim woman wears.  
From the transcript it is described as:

- **A visible sign of Muslim identity** – when you wear it, people recognize you as a Muslim woman.  
- **A protective garment** – it sends a clear message to anyone who might intend haram (unlawful or indecent) behavior, helping to repel such forces and keep you safe.  
- **A command from Allah** – it is legislated in the Qur’an to protect women from unwanted attention by unrelated men.  
- **A means of obedience and reward** – wearing it is seen as an act of kindness and mercy from Allah, not a restriction, and it brings spiritual benefit both in this life and the Hereafter.